In [1]:
from skimage.filters import gabor_kernel
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt 
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import math

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100
IMAGE_SIZE = 32
NUM_OF_CLASSES = 10

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100
IMAGE_SIZE = 32
NUM_OF_CLASSES = 10

def process_dataset(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
#     image = tf.image.per_image_standardization(image)
    
#     image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize images from 32x32 to 227x227
#     image = tf.image.resize(image, (IMAGE_SIZE,IMAGE_SIZE))

#     image = image/255.0
         
    label = tf.one_hot(tf.cast(label, tf.int32), NUM_OF_CLASSES)
        
    return image, label

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def construct_dataset(ds):
    ds = ds.shuffle(buffer_size=BATCH_SIZE)
    
#     ds=ds.repeat()
    ds = ds.batch(BATCH_SIZE, drop_remainder=False)
#     ds = ds.map(process_dataset,num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_data = construct_dataset(train_data)
test_data = construct_dataset(test_data)

steps_per_epoch=math.ceil(50000/BATCH_SIZE)
validation_steps=math.ceil(10000/BATCH_SIZE)


x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


In [4]:
# Importing the Keras libraries and packages
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam

# dimensionality of input and latent encoded representations
inpt_dim = (32, 32, 3)

inpt_img = Input(shape=inpt_dim)

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape = inpt_dim )(inpt_img)

# Block 1
cl1 = Conv2D(64, (3, 3), strides=(2, 2),activation='relu')(rescale)
bnl2 = BatchNormalization()(cl1)
# afl3 = Activation('relu')(bnl2)
pl4 = MaxPooling2D(pool_size = (2, 2))(bnl2)

# Adding a second convolutional layer
cl5 = Conv2D(128, (3, 3), strides=(1, 1), activation='relu')(pl4)
bnl6 = BatchNormalization()(cl5)
# afl7 = Activation('relu')(bnl6)
pl8 = MaxPooling2D(pool_size = (2, 2))(bnl6)
bnl9 = BatchNormalization()(pl8)

# Step 3 - Flattening
fl10 = Flatten()(bnl9)

# Step 4 - Full connection
dol11 = Dropout(0.5)(fl10)
dl12 = Dense(units = 256, activation = 'relu')(dol11)
dol13 = Dropout(0.2)(dl12)
dl14 = Dense(units = 64, activation = 'relu')(dol13)
dol15 = Dropout(0.1)(dl14)
output = Dense(units = 10, activation = 'sigmoid')(dol15)

classifier = Model(inpt_img, output)

# Compiling the CNN
opt = RMSprop(learning_rate=0.001)
# opt = Adam(learning_rate=0.01)

classifier.compile(optimizer = opt, loss ='binary_crossentropy', 
                   metrics = ['accuracy'])

print(classifier.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 15, 15, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 15, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 128)         73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 128)        

In [5]:
# Fitting the CNN to the images

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,  
                              min_delta=1e-4, mode='min', verbose=1)

stop_alg = EarlyStopping(monitor='val_loss', patience=35, 
                         restore_best_weights=True, verbose=1)



hist = classifier.fit(train_data,  epochs=1000, 
                   callbacks=[stop_alg, reduce_lr], 
                      validation_steps=validation_steps,
                      steps_per_epoch=steps_per_epoch,
                   validation_data=test_data)



Epoch 1/1000
500/500 [==============================] - 20s 40ms/step - loss: 0.2688 - accuracy: 0.3814 - val_loss: 0.2800 - val_accuracy: 0.3137
Epoch 2/1000
500/500 [==============================] - 22s 44ms/step - loss: 0.2205 - accuracy: 0.5021 - val_loss: 0.2509 - val_accuracy: 0.4502
Epoch 3/1000
500/500 [==============================] - 23s 46ms/step - loss: 0.2025 - accuracy: 0.5524 - val_loss: 0.1920 - val_accuracy: 0.5741
Epoch 4/1000
500/500 [==============================] - 24s 47ms/step - loss: 0.1926 - accuracy: 0.5752 - val_loss: 0.2035 - val_accuracy: 0.5404
Epoch 5/1000
500/500 [==============================] - 23s 47ms/step - loss: 0.1843 - accuracy: 0.5983 - val_loss: 0.1957 - val_accuracy: 0.5588
Epoch 6/1000
500/500 [==============================] - 25s 50ms/step - loss: 0.1786 - accuracy: 0.6124 - val_loss: 0.1978 - val_accuracy: 0.5583
Epoch 7/1000
500/500 [==============================] - 25s 51ms/step - loss: 0.1738 - accuracy: 0.6263 - val_loss: 0.1949 -

500/500 [==============================] - 62s 124ms/step - loss: 0.1020 - accuracy: 0.7936 - val_loss: 0.1265 - val_accuracy: 0.7367
Epoch 55/1000
500/500 [==============================] - 62s 123ms/step - loss: 0.1020 - accuracy: 0.7930 - val_loss: 0.1259 - val_accuracy: 0.7403
Epoch 56/1000
500/500 [==============================] - 63s 126ms/step - loss: 0.1005 - accuracy: 0.7977 - val_loss: 0.1257 - val_accuracy: 0.7419
Epoch 57/1000
500/500 [==============================] - 58s 116ms/step - loss: 0.1014 - accuracy: 0.7937 - val_loss: 0.1251 - val_accuracy: 0.7423
Epoch 58/1000
500/500 [==============================] - 55s 111ms/step - loss: 0.1009 - accuracy: 0.7970 - val_loss: 0.1256 - val_accuracy: 0.7425
Epoch 59/1000
500/500 [==============================] - 60s 119ms/step - loss: 0.0999 - accuracy: 0.7983 - val_loss: 0.1268 - val_accuracy: 0.7368
Epoch 60/1000
500/500 [==============================] - 57s 114ms/step - loss: 0.1002 - accuracy: 0.7974 - val_loss: 0.1250 -

500/500 [==============================] - ETA: 0s - loss: 0.0883 - accuracy: 0.8248
Epoch 00107: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
500/500 [==============================] - 58s 117ms/step - loss: 0.0883 - accuracy: 0.8248 - val_loss: 0.1235 - val_accuracy: 0.7526
Epoch 108/1000
500/500 [==============================] - 59s 118ms/step - loss: 0.0878 - accuracy: 0.8252 - val_loss: 0.1234 - val_accuracy: 0.7517
Epoch 109/1000
500/500 [==============================] - 58s 117ms/step - loss: 0.0879 - accuracy: 0.8255 - val_loss: 0.1242 - val_accuracy: 0.7531
Epoch 110/1000
500/500 [==============================] - 59s 119ms/step - loss: 0.0888 - accuracy: 0.8248 - val_loss: 0.1232 - val_accuracy: 0.7521
Epoch 111/1000
500/500 [==============================] - 60s 120ms/step - loss: 0.0875 - accuracy: 0.8240 - val_loss: 0.1240 - val_accuracy: 0.7536
Epoch 112/1000
500/500 [==============================] - 58s 116ms/step - loss: 0.0880 - accuracy: 0.824